In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax
jax.config.update('jax_platform_name', 'cpu')

In [3]:
from entot.data.data import create_gaussians, create_gaussian_split
from entot.plotting.plots import plot_1D
from entot.models.models import NoiseOutsourcingModel, KantorovichGapModel
from entot.models.utils import DataLoader
import matplotlib.pyplot as plt
import h5py
from tqdm import tqdm

import numpy as np
import jax
import jax.numpy as jnp
from entot.nets import UNet
import jax
import optax

In [4]:
def h5py_to_dataset(path, img_size=64, batch_size: int = 64):
    with h5py.File(path, "r") as f:
        # List all groups
        print("Keys: %s" % f.keys())
        a_group_key = list(f.keys())[0]

        # Get the data
        data = list(f[a_group_key])
    
        #dataset = 2 * jnp.transpose(jnp.array(data) / 255., (0, 3, 1, 2))- 1
        dataset = jnp.array(data) / 255.
        dataset = jax.image.resize(dataset, (len(dataset), img_size, img_size, 3), method='bilinear')    

    return DataLoader(dataset, batch_size=batch_size)

In [5]:
with jax.default_device(jax.devices("cpu")[0]):
    handbags = h5py_to_dataset("../../../../data/handbag_128.hdf5")

Keys: <KeysViewHDF5 ['imgs']>


In [6]:
with jax.default_device(jax.devices("cpu")[0]):
    shoes = h5py_to_dataset("../../../../data/shoes_128.hdf5")

Keys: <KeysViewHDF5 ['imgs']>


In [7]:
unet = UNet(diff_input_output=1)

In [8]:
state_unet = unet.create_train_state(jax.random.PRNGKey(0), optax.adam(1e-3), (64, 64, 3))

In [9]:
kg = KantorovichGapModel(epsilon=0.1, input_dim=[64, 64, 3], noise_dim=1, iterations=5000, neural_net=unet)

In [10]:
#b = handbags(jax.random.PRNGKey(0))
#b_noised = jnp.concatenate((b, jax.random.normal(jax.random.PRNGKey(0), (64, 64, 64,1))), axis=-1)
# transported = kg.state_neural_net.apply_fn({"params": kg.state_neural_net.params}, b_noised)

In [11]:
kg(handbags, shoes)

  3%|▎         | 169/5000 [32:47<15:25:40, 11.50s/it]